Objective: say we read in a parititioned i94 dataset (by `i94yr` and `i94mon`), would will still be able to use WHERE for this?

(answer: no. We will have to read those paritions individually like this: https://stackoverflow.com/questions/47191078/spark-sql-queries-on-partitioned-data-using-date-ranges)

In [1]:
import os
import pyspark
import configparser
import pandas as pd
import pyspark.sql.functions as F
import pyspark.sql.types as T

from datetime import datetime, timedelta
from pyspark.sql import SparkSession

In [7]:
# Ensure Jupyter Notebook display pandas dataframe fully. Show all columns. Do not truncate column value.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', 1000)

# Dev config from the non-secret configuration file
config_dev = configparser.ConfigParser()
config_dev.read_file(open('aws_dev.cfg'))

PAR_I94_FILE_BY_I94YR_I94MON = config_dev.get('DATA_PATHS_LOCAL', 'PAR_I94_FILE_BY_I94YR_I94MON')

In [3]:
spark = SparkSession.builder.\
    config("spark.jars.repositories", "https://repos.spark-packages.org/").\
    config("spark.jars.packages", "saurfang:spark-sas7bdat:2.0.0-s_2.11").\
    enableHiveSupport().getOrCreate()

In [8]:
df_i94 = spark.read.parquet(PAR_I94_FILE_BY_I94YR_I94MON)

In [9]:
# Register the spark dataframe as a spark SQL data for ease of adhoc querying.
df_i94.createOrReplaceTempView("df_i94")

In [10]:
df_i94_1 = spark.sql(f"""\
    select count(*) as records
    from df_i94
    where (i94yr = 2016) and (i94mon = 3)
""")

AnalysisException: "cannot resolve '`i94yr`' given input columns: [df_i94.entdepd, df_i94.biryear, df_i94.arrdate, df_i94.occup, df_i94.i94port, df_i94.cicid, df_i94.i94mode, df_i94.entdepa, df_i94.visapost, df_i94.airline, df_i94.i94visa, df_i94.i94addr, df_i94.admnum, df_i94.dtaddto, df_i94.visatype, df_i94.i94bir, df_i94.i94res, df_i94.fltno, df_i94.i94cit, df_i94.depdate, df_i94.insnum, df_i94.count, df_i94.entdepu, df_i94.matflag, df_i94.dtadfile, df_i94.gender]; line 3 pos 11;\n'Aggregate [count(1) AS records#52L]\n+- 'Filter (('i94yr = 2016) && ('i94mon = 3))\n   +- SubqueryAlias `df_i94`\n      +- Relation[cicid#0,i94cit#1,i94res#2,i94port#3,arrdate#4,i94mode#5,i94addr#6,depdate#7,i94bir#8,i94visa#9,count#10,dtadfile#11,visapost#12,occup#13,entdepa#14,entdepd#15,entdepu#16,matflag#17,biryear#18,dtaddto#19,gender#20,insnum#21,airline#22,admnum#23,... 2 more fields] parquet\n"

In [11]:
spark.stop()